<a href="https://colab.research.google.com/github/IdaWiweka/datathon/blob/IdaWiweka/Datathon_DataProcessing_4_extraction_grouped.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from huggingface_hub import login
from google.colab import userdata
huggingface_token = userdata.get('huggingface_llama32')

# Login using the token
login(huggingface_token)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import pandas as pd
from google.colab import files
import json
import torch
from transformers import pipeline
from tqdm import tqdm
# Load data from URL
#url = "https://raw.githubusercontent.com/CBDRH/hds-datathon-data/refs/heads/main/release/epiwatch-latest.csv"
#data = pd.read_csv(url)
# Define the folder path
folder_path = '/content/drive/My Drive/processed_batches/Dataset_id'
file_path = '/content/drive/My Drive/processed_batches/Dataset_id/content_extraction_1.csv'

# Read the CSV file
file_name = "epiwatch-latest.csv"

data = pd.read_csv(os.path.join(folder_path, file_name))




# Add an ID column for unique identification
#data['id'] = range(1, len(data) + 1)

# Filter the rows where the "country" column equals "Nigeria"
df = data[data['country'] == "China"]
#df = data[data['diseases_original'].str.contains("COVID-19|corona|covid19", case=False, na=False)]
#df = data[data['diseases_original'].str.contains("Influenza", case=False, na=False)]
#df = data[data['diseases_original'].str.contains("Legionnaires", case=False, na=False)]
#df = data[data['diseases_original'].str.contains("Cholera", case=False, na=False)]
#df = data[data['diseases_original'].str.contains("Dengue", case=False, na=False)]
#df = data[data['diseases_original'].str.contains("Mpox", case=False, na=False)]
#df = data[data['diseases_original'].str.contains("Measles", case=False, na=False)]
# Select a subset of rows (1 to 20)
#df = data.iloc[:50]

# Process publication-date and create year_month column
df['publication-date'] = pd.to_datetime(df['publication-date'], errors='coerce')
df['year_month'] = df['publication-date'].dt.to_period('M')

# Define function to process each group
def process_group(group_key, group_df):
    print(f"Processing group: {group_key}")
    result = group_df[['publication-date', 'country', 'diseases_original','syndrome_title', 'title', "Translated_Content"]].to_dict(orient='records')
    print("Group Result:", result)
    return result

# Define parameters for the text generation pipeline
parameters = {
    "model_name": "meta-llama/Llama-3.2-3B-Instruct",
    "device": None,
    "torch_dtype": torch.bfloat16,
    "temperature": 0.2,
    "top_p": 0.8,
    "max_new_tokens": 1024,

}

# Define the main extraction function
def extract_information_from_dataframe(
    dataframe,
    model_name,
    device,
    torch_dtype,
    temperature,
    top_p,
    max_new_tokens

):
    # Determine the device (CUDA or CPU)
    device = device or ("cuda" if torch.cuda.is_available() else "cpu")

    # Initialize the text generation pipeline
    generator = pipeline("text-generation", model=model_name, device=device, torch_dtype=torch_dtype)

    # Group the data
    grouped = dataframe.groupby(['country', 'diseases_original', 'year_month'])

    # Store results
    results = []

    # Process each group
    for group_key, group_df in tqdm(grouped, total=len(grouped), desc="Processing groups"):
        # Process the group into structured data
        result = process_group(group_key, group_df)

        # Prepare the prompt
        prompt = [
                {
                "role": "system",
                "content": """
                            You are an epidemiologist tasked with analyzing unstructured data to identify relationships of the data based on the disease or syndrome.

                            Follow these steps:
                            1. Analyze whether the disease occurred in the same timeframe within the same country (monthly).
                            2. Identify the syndrome such as Acute gastroenteritis; Severe acute respiratory syndrome; Febrile syndromes; Pneumonia; Influenza-like illness.
                            3. Extract the start and end dates of the disease from the data.
                            4. Calculate the number of new cases, ensuring duplicates are excluded. The final output is integer.
                            5. Calculate the number of deaths (mortality), ensuring duplicates are excluded. The final output is integer.
                            6. Count the total number of unique news reports from title column.
                            7. Determine whether the disease qualifies as an outbreak, epidemic, or pandemic based on WHO definitions.
                            8. Summarize your findings and output the results in the specified format. Do not rewrite the steps or provide explanations.

                            Output JSON Format:
                            {
                              "country": "country_name",
                              "disease": "disease_name",
                              "syndrome": "syndrome_name",
                              "date_started": "YYYY-MM-DD",
                              "date_end": "YYYY-MM-DD",
                              "number_of_new_cases": "number_of_new_cases",
                              "number_of_mortality": "number_of_mortality",
                              "total_report": "total_count_of_news",
                              "status": "outbreak/epidemic/pandemic"
                             }

                            Return your response in JSON format without any additional explanation.""",
                },
                {
                    "role": "user",
                    "content": f"""Here is the data:
                                  "{result}"
                                   Do not rewrite here is the data.""",
                },
            ]

        # Generate the summary using the pipeline
        generation = generator(
            prompt,
            do_sample=True,
            temperature=temperature,
            top_p=top_p,
            max_new_tokens=max_new_tokens
        )

        # Extract the generated text
        assistant_response = generation[0]["generated_text"]
        assistant_content = assistant_response[2]['content']

        # Append the result

        results.append({

            "extracted_info": assistant_content
        })

    return results

# Call the function and process the data
results = extract_information_from_dataframe(
    dataframe=df,
    **parameters
)

# Print results in JSON format
print(results)


<ipython-input-3-6cf664618d29>:38: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['publication-date'] = pd.to_datetime(df['publication-date'], errors='coerce')
<ipython-input-3-6cf664618d29>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['publication-date'] = pd.to_datetime(df['publication-date'], errors='coerce')
<ipython-input-3-6cf664618d29>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

...

Processing group: ('China', 'covid19', Period('2022-03', 'M'))
Group Result: [{'publication-date': Timestamp('2022-03-01 00:00:00'), 'country': 'China', 'diseases_original': 'covid19', 'syndrome_title': nan, 'title': 'Chinese mainland 75 new cases of local confirmed cases were added to the Epidemic Bulletin, and 96 cases were imported from Beijing, Shanghai and Guangdong, and 84 cases were imported from Hong Kong | Reuters', 'Translated_Content': 'Error 401: Unable to fetch content'}, {'publication-date': Timestamp('2022-03-01 00:00:00'), 'country': 'China', 'diseases_original': 'covid19', 'syndrome_title': nan, 'title': 'On the 28th, 4 new cases of positive infection in Tianjin were found to be screened at centralized isolation points - Xinhua Daily;', 'Translated_Content': 'The reporter learned from the Municipal Prevention and Control Command that from 0:00 on February 28th to 24th, Tianjin reported that the test results of the sample nucleic acid detection of four new crown virus n

Token indices sequence length is longer than the specified maximum sequence length for this model (137981 > 131072). Running this sequence through the model will result in indexing errors
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (131072). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.
Processing groups:  22%|██▏       | 51/236 [10:52<1:51:01, 36.01s/it]

Processing group: ('China', 'covid19', Period('2022-04', 'M'))
Group Result: [{'publication-date': Timestamp('2022-04-01 00:00:00'), 'country': 'China', 'diseases_original': 'covid19', 'syndrome_title': nan, 'title': 'There are BA.2 Sub-variants, Asian Covid-19 Infections 100 Million Cases - CNBC Indonesia', 'Translated_Content': 'Jakarta, CNBC Indonesia - The Covid -19 case has passed 100 million on Wednesday (3/30/2022).This is based on Reuters reports, where Asia also records the rise of cases due to the dominance of the sub-variant Omicron BA.2.Reuters reported Asia reported more than 1 million COVID-19 cases per two days.Asia has also contributed 21% of all COVID-19 cases, quoted Wednesday (3/30/2022).Sub-variant BA.2 has pushed the number of cases to the highest level in recent weeks.This happened to South Korea, China and Vietnam.The World Health Organization or WHO reports BA.2 represents almost 86% of all sequential cases.South Korea leads the average daily number of cases rep

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Processing groups:  22%|██▏       | 51/236 [11:22<41:16, 13.39s/it]  


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.82 GiB. GPU 0 has a total capacity of 39.56 GiB of which 458.81 MiB is free. Process 8046 has 39.11 GiB memory in use. Of the allocated memory 32.18 GiB is allocated by PyTorch, and 6.43 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
print(json.dumps(results, indent=4))

NameError: name 'results' is not defined

In [ ]:
import pandas as pd
import json

# Create a DataFrame
df = pd.DataFrame(results)

# Parse the JSON in the 'extracted_info' column
def parse_extracted_info(row):
    try:
        parsed = json.loads(row)
        # Handle cases where parsed is a list of dictionaries
        if isinstance(parsed, list):
            return parsed  # Keep as list
        return [parsed]  # Convert single dictionary to list
    except json.JSONDecodeError:
        return None  # Handle invalid JSON gracefully

df['parsed_info'] = df['extracted_info'].apply(parse_extracted_info)

# Flatten the list of dictionaries into separate rows
expanded_rows = []
for index, row in df.iterrows():
    for item in row['parsed_info']:
        expanded_rows.append(item)

# Convert the expanded rows into a DataFrame
parsed_df = pd.DataFrame(expanded_rows)

print(parsed_df)
# Save and download the file
parsed_df.to_csv('Vietnam_grouped_extraction.csv', index=False)
files.download('Vietnam_grouped_extraction.csv')

TypeError: 'NoneType' object is not iterable

In [ ]:
import pandas as pd
import json

# Create a DataFrame
df = pd.DataFrame(results)

# Parse the JSON in the 'extracted_info' column
def parse_extracted_info(row):
    try:
        parsed = json.loads(row)
        # Handle cases where parsed is a list of dictionaries
        if isinstance(parsed, list):
            return parsed  # Keep as list
        return [parsed]  # Convert single dictionary to list
    except json.JSONDecodeError:
        return []  # Handle invalid JSON by returning an empty list to avoid the error

df['parsed_info'] = df['extracted_info'].apply(parse_extracted_info)

# Flatten the list of dictionaries into separate rows
expanded_rows = []
for index, row in df.iterrows():
    # Check if row['parsed_info'] is iterable before iterating
    if row['parsed_info']:
        for item in row['parsed_info']:
            expanded_rows.append(item)

# Convert the expanded rows into a DataFrame
parsed_df = pd.DataFrame(expanded_rows)

print(parsed_df)
# Save and download the file
parsed_df.to_csv('Vietnam_grouped_extraction.csv', index=False)
files.download('Vietnam_grouped_extraction.csv')

    country                 disease                syndrome date_started  \
0   Vietnam             Influenza A   Acute gastroenteritis   2022-06-09   
1   Vietnam             Influenza A               Pneumonia   2022-09-09   
2   Vietnam             Influenza A  Influenza-like illness   2022-10-08   
3   Vietnam             Influenza A  Influenza-like illness   2022-12-08   
4   Vietnam                    Mpox   Acute gastroenteritis   2022-01-11   
..      ...                     ...                     ...          ...   
95  Vietnam             influenza b         Avian influenza   2021-08-02   
96  Vietnam             influenza b    Respiratory diseases   2022-07-11   
97  Vietnam               influenza  Influenza-like illness   2021-04-03   
98  Vietnam  influenza, listeriosis  Influenza-like illness   2021-06-07   
99  Vietnam                 measles   Acute gastroenteritis   2022-10-11   

      date_end number_of_new_cases number_of_mortality total_report    status  
0      

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
def parse_extracted_info(info):
    try:
        # Remove backticks and strip any extra spaces
        cleaned_info = info.strip("```").strip()
        # Convert the string to a dictionary
        return json.loads(cleaned_info)
    except json.JSONDecodeError:
        return {}

# Create a DataFrame
df = pd.DataFrame(results)

# Parse the 'extracted_info' column into dictionaries
df['parsed_info'] = df['extracted_info'].apply(parse_extracted_info)

# Expand the parsed_info dictionary into separate columns
parsed_df = pd.json_normalize(df['parsed_info'])



print(parsed_df)

# Save and download the file
parsed_df.to_csv('Indonesia_grouped_extraction.csv', index=False)
files.download('Indonesia_grouped_extraction.csv')
# Save the DataFrame to a CSV file
#parsed_df.to_csv(file_path, index=False)



AttributeError: 'list' object has no attribute 'keys'

In [ ]:
import json

# Function to process extracted_info and structure into JSON
def process_extracted_info(data_list):
    structured_data = []
    for item in data_list:
        try:
            extracted_info = item["extracted_info"]
            lines = extracted_info.split("\n")
            entry = {}
            for line in lines:
                if ":" in line:
                    key, value = line.split(":", 1)
                    key = key.strip().strip("'")
                    value = value.strip().strip("'")
                    if key == "number_of_new_cases":
                        # Handle multiple cases
                        value = value.split(", ")
                    entry[key] = value
            structured_data.append(entry)
        except Exception as e:
            print(f"Error processing entry: {e}")
    return structured_data

# Process and print structured JSON
structured_output = process_extracted_info(results)
print(json.dumps(structured_output, indent=4))


[
    {
        "\"country\"": "\"Nigeria\",",
        "\"disease\"": "\"cholera\",",
        "\"syndrome\"": "\"Acute gastroenteritis\",",
        "\"date_started\"": "\"2020-10-17\",",
        "\"date_end\"": "\"2020-10-17\",",
        "\"number_of_new_cases\"": "\"39,795\",",
        "\"number_of_mortality\"": "\"0\",",
        "\"total_report\"": "\"1\",",
        "\"status\"": "\"outbreak\""
    },
    {
        "\"country\"": "\"Nigeria\",",
        "\"disease\"": "\"cholera\",",
        "\"syndrome\"": "\"Acute gastroenteritis\",",
        "\"date_started\"": "\"2020-11-10\",",
        "\"date_end\"": "\"2020-11-19\",",
        "\"number_of_new_cases\"": "\"10\",",
        "\"number_of_mortality\"": "\"8\",",
        "\"total_report\"": "\"2\",",
        "\"status\"": "\"epidemic\""
    },
    {
        "\"country\"": "\"Nigeria\",",
        "\"disease\"": "\"cholera\",",
        "\"syndrome\"": "\"Acute gastroenteritis\",",
        "\"date_started\"": "\"2021-01-11\",",
       